In [1]:
import pandas as pd
import numpy as np
import os
import json
import ast
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
english_stop_words = list(stopwords.words('english'))
import gensim.corpora as corpora
import gensim
from gensim.utils import simple_preprocess
from pprint import pprint
import pyLDAvis.gensim
import pickle
import pyLDAvis
import time
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pi96a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pi96a\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [84]:
path = "C:/Drive D/Datasets/Aminer Data"
os.chdir(path)

In [85]:
topic_words_df = pd.read_csv("new_dblp-ref-3 LDA topic words.csv")

In [86]:
topic_words_df.drop(['Unnamed: 0'], axis = "columns", inplace = True)

In [87]:
doc_topic_df = pd.read_csv("new_dblp-ref-3 doc-topic-df.csv")

In [88]:
%%time
# Removing all the Null values from the dataset
df = pd.read_json("new_dblp-ref-3.json")
# drop all NA values from the dataset
print("Before Dropping NA : ", df.shape)
df = df.dropna().reset_index(drop = True)
print("After Dropping NA : ", df.shape)

Before Dropping NA :  (79007, 8)
After Dropping NA :  (35174, 8)
CPU times: total: 2.16 s
Wall time: 2.98 s


In [97]:
df = pd.concat([doc_topic_df[['Topic']], df], axis = "columns")

In [102]:
sub_df = df.sample(2000, random_state = 23).reset_index(drop = True)

In [107]:
sub_df.head()

,Topic,abstract,authors,n_citation,references,title,venue,year,id
0,1,The open-source software project OpenPhase all...,"[Marvin Tegeler, Oleg Shchyglo, Reza Darvishi ...",0,"[35d0c371-6a04-49a1-822d-16365ec0b4ef, b3dd5f8...",Parallel multiphase field simulations with Ope...,Computer Physics Communications,2017,a4f44262-95d0-4550-a0bf-56b8cf55b2e2
1,3,Wireless sensor networks with radio frequency ...,"[Vojislav B. Misic, Jelena V. Misic, Mohammad ...",0,"[15cb7fd8-134a-4c30-a579-286a685a320d, 1aee33c...",Optimum Zoning in RF-Recharged Sensor Networks,vehicular technology conference,2016,492121d5-7ef4-490b-956b-610ec45a38e8
2,4,This research was performed to investigate the...,"[Majid Memarian Sorkhabi, Javad Frounchi, Parv...",0,"[05be8ba6-2e04-4f03-a310-b5dd9997aa4e, 26c5aec...",Deep-Brain Transcranial Stimulation: A Novel A...,IEEE Access,2017,fee7ef89-1665-4b8b-927d-b5fd80194002
3,1,The performance of stereophonic acoustic echo ...,"[Maria Luis Valero, Ilkay Yildiz, Edwin Maband...",0,"[1377b277-655a-4b44-9181-2e28c12aa5a4, 5528066...",Coherence-aware stereophonic residual echo est...,,2017,db0d91ec-9297-4493-8df2-9443a3929204
4,5,Web-based Social Networks (W-bSNs) have recent...,"[Yecely Aridaí Díaz-Beristain, Guillermo-de-Je...",0,[3c0c7536-7f11-4e63-bf6d-32d67050d8ef],Strategies for Growing User Popularity through...,advances in multimedia,2017,8e58438d-5c8f-4e4e-890b-ed71a8172a8b


In [103]:
len(set((chain.from_iterable(sub_df["authors"].values.tolist()))))

7282

In [105]:
unique_authors = set((chain.from_iterable(sub_df["authors"].values.tolist())))

In [106]:
author_coauthor_matrix = dict()

for author_1 in unique_authors:
    for author_2 in unique_authors:
        if author_1 != author_2:
            if set([author_1, author_2]).intersection()

{'Zhichao Song',
 'Chaojie Zhang',
 'Jane G. Payumo',
 'Manolis Maragoudakis',
 'M. Manikandan',
 'Wen Hu',
 'Sarah Latus',
 'S. Mhira',
 'Lifeng Sun',
 "Sidney K. D'Mello",
 'QingJun Song',
 'Wenhuan Lu',
 'Atsushi Harada',
 'Charilaos Akasiadis',
 'Haobin Jiang',
 'David Lonie',
 'Jinhee Lee',
 'Orcun Goksel',
 'Xueming Yan',
 'Niki Kyriakou',
 'Jie Shao',
 'Zoubin Ghahramani',
 'Hideki Ogawa',
 'Huan Wang',
 'Irina Illina',
 'Xiao-Shun Zhou',
 'Zhiyong Zhang',
 'Min Xiang',
 'Majing Su',
 'Wei Dong',
 'Lajos Hanzo',
 'Csongor Mark Horvath',
 'Ping Huang',
 'Daniel Schaffer Ferreira Jorge',
 'Debbie Marr',
 'Neil D. Lawrence',
 'Cheng-An Wu',
 'Yuanyi Xue',
 'Francis Chow',
 'Jae-Jin Kim',
 'G. Seshikala',
 'Tianxiang Li',
 'Ling He',
 'Furkan Gürpınar',
 'Eric Smith',
 'Adrien Escande',
 'Moti Yung',
 'Anton V. Prokhorov',
 'A. S. Namitha',
 'Christophe Dubach',
 'Borivoje Nikolic',
 'Xiang Xia',
 'Kun Jia',
 'Katrin Panzenböck',
 'Ananthram Swami',
 'Mohammad Reza Yousefi',
 'Alexa